# Goal

get metadata and PCs for power calculation


# Var

In [1]:
work_dir = 'Gut-microbiota-in-Psoriasis/power_analysis'

# Init

In [3]:
library(microbiome)
library(dplyr)
library(phyloseq)
library(ggplot2)
library(plyr)
library(zCompositions)
library(compositions)
library(textshape)
library(tibble)
library(tidyverse)


microbiome R package (microbiome.github.com)
    


 Copyright (C) 2011-2022 Leo Lahti, 
    Sudarshan Shetty et al. <microbiome.github.io>



Attaching package: 'microbiome'


The following object is masked from 'package:ggplot2':

    alpha


The following object is masked from 'package:base':

    transform



Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


------------------------------------------------------------------------------

You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)

------------------------------------------------------------------------------


Attaching package: 'plyr'


The following objects are masked from 'package:dplyr':

    arrange, count, desc, failwith, id, mutate, rename,

# Load

In [3]:
load('../psq3.RData')

### Function

In [2]:
make_pca <- function(psq, tax_rank) {
    psq_rank <- aggregate_taxa(psq, level = tax_rank)
    
    # STEP 1:
    # replace 0 values with the count zero multiplicative method
    # this function expects the samples to be in rows
    otu_rank_CZM <- t(cmultRepl(t(otu_table(psq_rank)), method="CZM", output="p-counts"))
    
    # CONVERT to PROPORTIONS by sample (columns) using the apply function
    otu_rank_CZM_prop <- apply(otu_rank_CZM, 2, function(x){x/sum(x)})
    
    # STEP 2: CLR Transformation:
    clr_rank <- t(apply(otu_rank_CZM_prop, 2, function(x){log(x) - mean(log(x))}))

    # NOTE: sample_id order will be identical at aggregated phyloseq objects

    # 1) take the transformed abundance table, 
    # 2) extract row-names, i.e. sample ids:
    # 3) find matching ids for psoriasis patients, assign 'green4' if match OR 'red' if not - controls

    mycols=data.frame(c(ifelse(rownames(clr_rank) %in% psor_id,'red','green4')))

    colnames(mycols) <- "colors"

    # Singular value decompositon: PCA 
    pcx_rank <- prcomp(clr_rank)

    # Sum the total variance
    d.mvar <- sum(pcx_rank$sdev^2)

    # modify donor IDS - rownames - make shorter to look nice on the plot:
    row.names(pcx_rank$x) <- gsub("_profile","", row.names(pcx_rank$x))
    output <- list(pcx_rank, mycols, d.mvar)

    return(output)
}

# Analysis

In [6]:
psq3

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 249 taxa and 100 samples ]
sample_data() Sample Data:       [ 100 samples by 56 sample variables ]
tax_table()   Taxonomy Table:    [ 249 taxa by 7 taxonomic ranks ]

In [21]:
ord_pty <- ordinate(psq3, "PCoA", "bray",weighted=FALSE)

In [9]:
id_diagnosis<-sample_data(psq3)[,c('ID','Diagnosis')]
psor_id <- c(id_diagnosis[id_diagnosis$Diagnosis=='psoriasis',1])$ID

species <- make_pca(psq3, "Species")
pcx_species <- species[[1]]
mycols_species <- species[[2]]
d.mvar_species <- species[[3]]

Warning message in cmultRepl(t(otu_table(psq_rank)), method = "CZM", output = "p-counts"):
"Column no. 3 containing >80% zeros/unobserved values deleted (see arguments z.warning and z.delete).
Column no. 5 containing >80% zeros/unobserved values deleted (see arguments z.warning and z.delete).
Column no. 6 containing >80% zeros/unobserved values deleted (see arguments z.warning and z.delete).
Column no. 9 containing >80% zeros/unobserved values deleted (see arguments z.warning and z.delete).
Column no. 10 containing >80% zeros/unobserved values deleted (see arguments z.warning and z.delete).
Column no. 13 containing >80% zeros/unobserved values deleted (see arguments z.warning and z.delete).
Column no. 16 containing >80% zeros/unobserved values deleted (see arguments z.warning and z.delete).
Column no. 23 containing >80% zeros/unobserved values deleted (see arguments z.warning and z.delete).
Column no. 26 containing >80% zeros/unobserved values deleted (see arguments z.warning and z.del

In [13]:
str(pcx_species)

List of 5
 $ sdev    : num [1:100] 16 13.7 12.4 10.8 10.4 ...
 $ rotation: num [1:137, 1:100] -0.1209 -0.0421 0.079 0.0813 0.1041 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:137] "Methanobrevibacter_smithii" "Bifidobacterium_adolescentis" "Bifidobacterium_bifidum" "Bifidobacterium_longum" ...
  .. ..$ : chr [1:100] "PC1" "PC2" "PC3" "PC4" ...
 $ center  : Named num [1:137] -2.67 2.06 -2.71 1.79 -2.54 ...
  ..- attr(*, "names")= chr [1:137] "Methanobrevibacter_smithii" "Bifidobacterium_adolescentis" "Bifidobacterium_bifidum" "Bifidobacterium_longum" ...
 $ scale   : logi FALSE
 $ x       : num [1:100, 1:100] 18.409 -5.757 4.608 0.754 -6.64 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:100] "D9" "D99" "D98" "D97" ...
  .. ..$ : chr [1:100] "PC1" "PC2" "PC3" "PC4" ...
 - attr(*, "class")= chr "prcomp"


In [15]:
round(sum(pcx_species$sdev[1]^2)/d.mvar_species, 3)
round(sum(pcx_species$sdev[2]^2)/d.mvar_species, 3)
round(sum(pcx_species$sdev[3]^2)/d.mvar_species, 3)
print(0.084 + 0.061 + 0.05)

[1] 0.084

[1] 0.061

[1] 0.05

[1] 0.195


In [16]:
# Important: add '\t' in the beggining of the header line
id_diagnosis <- sample_data(psq3)[,c('ID','Diagnosis', 'Calprotectin')]

In [17]:
class(id_diagnosis) <- c("data.frame")

In [18]:
id_diagnosis[1:2,]

,ID,Diagnosis,Calprotectin
,<chr>,<chr>,<dbl>
D9_profile,D9_profile,control,13.25
D99_profile,D99_profile,control,17.75


In [25]:
pcx_species$x

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC91,PC92,PC93,PC94,PC95,PC96,PC97,PC98,PC99,PC100
D9,18.4085826,-16.33812985,13.11284058,-0.1710436,0.0317828,-2.8362844,9.84427563,-13.4904168,16.9617219,-14.7670471,...,-1.54444465,-0.78309254,-1.36625338,2.30866697,0.090136147,-0.34890508,-0.54284248,-0.36369658,0.285584149,5.309133e-15
D99,-5.7567416,-9.54746711,-16.90779993,11.9573567,10.3560796,-13.0170990,-9.15799018,-3.2474509,-10.5960498,-5.6557453,...,1.36571428,-1.24567664,1.29839632,1.41724572,1.433588965,-1.35701705,0.50558395,-0.35508806,0.432919168,-8.491779e-16
D98,4.6082042,16.62194631,0.15790946,1.7185819,0.1586647,1.8644503,-23.00894655,9.5633085,-1.9308884,4.3021551,...,0.44853912,-0.94556822,0.05975688,0.90099166,0.143842529,0.92077859,-0.37830042,-0.06295333,1.046262191,2.696121e-15
D97,0.7539063,-3.63487739,-6.16918845,-0.8010707,-4.5408345,22.0150645,5.64111249,-3.1174533,-0.4642387,8.6321876,...,2.80544271,-0.14821369,-0.39133990,-0.61446117,-0.857844483,0.59585135,0.43403043,-1.36698118,0.316783464,5.627641e-15
D96,-6.6397691,-0.01060802,-11.06681657,12.0742497,-7.6269125,9.9038087,4.37607021,15.3185066,-16.3508437,-5.8770771,...,0.02629803,0.35338263,0.05746113,-0.55060134,1.199081813,0.03749973,-0.75077769,1.22113357,-0.394527391,1.043331e-14
D95,-1.6474271,-20.68261609,-14.06518359,5.0669925,-7.1106278,-0.6085964,12.66511750,-6.2833494,9.8573340,0.4905765,...,0.46766397,-1.71966112,-0.29504429,-0.78896322,-0.081820972,0.65190977,1.78859183,0.51722570,0.015061981,5.727807e-15
D94,15.8046246,-1.69749624,-11.21017395,1.8355642,-7.8209270,15.1501630,12.42895811,-0.6834428,-0.9487619,-13.3382367,...,0.85170539,1.38983094,0.47691075,1.31527779,0.278186423,-0.63118952,-1.07773124,-0.33082765,-0.274631728,9.241269e-15
D93,-28.6905604,-5.92550998,-1.46062487,10.7088036,-0.1096841,2.7643515,2.49940880,-3.0050492,5.3771616,-2.4458915,...,-0.72727638,0.56636443,0.08591891,-1.00716142,-0.796702260,-0.29672231,-0.28925378,1.64425205,-0.908413537,5.045801e-15
D92,-11.2913104,-16.05603209,-1.67683868,6.6396029,1.1244510,6.9825549,-16.41697015,7.5031236,-6.6711828,-11.4615317,...,-1.53176158,-2.05432532,-1.14257466,0.59191143,0.810616508,0.08104826,0.51245090,0.19128508,0.573765877,9.886544e-15
D91,3.4915347,-4.75661147,3.09448751,11.5324748,2.7567919,4.6389105,1.76463880,-11.5392544,-11.5092131,-2.0587679,...,1.66922142,-1.21624155,0.16047592,-0.54340781,-0.782706775,-0.73325429,0.45214070,0.19744078,0.005102853,5.907841e-15


In [26]:
PC_coords <- pcx_species$x

In [8]:
eigs_pty<- ord_pty$values
sum(eigs_pty$Eigenvalues[1:4])/sum(eigs_pty$Eigenvalues)*100

# top 4 PCs explain 52% of pathway variation

[1] 51.93623

In [9]:
# get and export the bray distance matrix used to compute PCoA:
bray_d <- phyloseq::distance(physeq=psq3, method="bray")

In [23]:
ord_pty$vectors

,Axis.1,Axis.2,Axis.3,Axis.4,Axis.5,Axis.6,Axis.7,Axis.8,Axis.9,Axis.10,...,Axis.52,Axis.53,Axis.54,Axis.55,Axis.56,Axis.57,Axis.58,Axis.59,Axis.60,Axis.61
D9_profile,0.17058585,-0.001363179,0.05577144,-0.11803767,-0.145413016,-0.0947404898,0.0471785255,-0.126873437,0.119014481,-0.131568651,...,-0.0108730397,4.251234e-02,-3.480850e-03,-0.003410827,1.657127e-02,0.0052003398,0.003144283,1.513349e-03,0.0048374172,-6.850708e-04
D99_profile,-0.16300604,-0.212192527,0.19710386,0.06483316,-0.031422427,0.0556380907,-0.0389929002,0.041637666,0.061888938,-0.084199414,...,0.0056384388,1.300711e-02,-5.104414e-04,0.033208819,-6.241207e-03,-0.0129871243,-0.007902837,1.150236e-03,-0.0020082110,-6.623643e-04
D98_profile,-0.03264002,0.109935642,0.16117134,0.14285781,0.188699397,0.0207096167,-0.0096481061,0.132178601,-0.045228446,-0.038178077,...,0.0255752571,-2.558119e-03,1.736995e-02,0.020894204,1.230973e-04,-0.0205414892,-0.003220522,9.316604e-03,0.0104701163,-1.543238e-04
D97_profile,0.33175367,0.214145040,0.08121568,0.04289116,0.054371987,0.0311604294,-0.0452735309,0.166243332,-0.033821518,0.017162686,...,-0.0183202795,1.253395e-02,-1.697497e-02,-0.019716224,-2.109079e-02,-0.0092961755,-0.003576432,7.277265e-03,0.0273908680,6.801170e-04
D96_profile,0.05430690,-0.302469521,0.02287812,-0.14329423,-0.071646785,-0.0919748212,-0.0480383665,0.196084605,0.055360870,-0.029175590,...,0.0286418506,5.581735e-03,6.540118e-04,0.006545980,6.183413e-03,-0.0037715665,-0.002227332,-8.364633e-03,-0.0044092948,7.427375e-04
D95_profile,0.28858169,0.026591873,0.03752251,0.14632148,-0.035853451,-0.0555187079,-0.1008373587,-0.008515318,-0.038314141,-0.058763503,...,0.0217406664,-4.006399e-02,-9.187557e-03,0.013868940,3.582383e-02,0.0105923961,-0.005734696,-8.808676e-03,0.0051017522,-9.395966e-04
D94_profile,0.18819541,-0.129103900,-0.14124922,-0.01586263,-0.187050511,0.0291700385,0.0138528468,0.002420364,0.069612057,0.161645414,...,-0.0111153604,3.111073e-02,-1.446152e-02,-0.011633603,-1.127549e-02,0.0041715497,-0.007440350,2.723918e-03,-0.0055884578,9.854146e-04
D93_profile,-0.31385439,-0.132470204,0.14258856,-0.02789048,0.052401029,0.0792306883,0.0322252100,-0.065822341,0.065866223,0.009963554,...,-0.0226921203,4.645438e-03,-1.937159e-03,0.029216341,-2.031723e-02,0.0268736508,0.006106241,4.162607e-03,0.0038274069,3.795078e-04
D92_profile,-0.02387078,-0.149338383,0.09247192,0.17336910,0.200281379,-0.0173428649,0.0340853656,0.115313912,-0.034545240,-0.071353563,...,-0.0083540338,3.397401e-02,-2.903892e-02,-0.016304519,1.874721e-02,0.0103947370,-0.019365233,8.712558e-04,-0.0131561678,4.242072e-04
D91_profile,0.14265361,-0.179277802,-0.06487867,0.08417172,-0.318962228,0.0082895738,0.0798314701,-0.011522425,0.052956023,-0.024103216,...,-0.0242554374,-2.001418e-02,4.393815e-03,0.009270397,6.285792e-03,-0.0113412796,0.013210535,8.018631e-05,-0.0045356961,-5.944591e-04


In [13]:
PC_coords <- ord_pty$vectors

In [27]:
ids_pcs <- data.frame(IDs=c(row.names(PC_coords)),PC_coords[,1:4])

In [34]:
ids_pcs[1:3,]
ids_pcs <- ids_pcs %>% mutate(IDs = paste0(IDs, "_profile"))


,IDs,PC1,PC2,PC3,PC4
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
D9,D9,18.408583,-16.338130,13.1128406,-0.1710436
D99,D99,-5.756742,-9.547467,-16.9077999,11.9573567
D98,D98,4.608204,16.621946,0.1579095,1.7185819


In [31]:
id_diagnosis[1:3,]

,ID,Diagnosis,Calprotectin
,<chr>,<chr>,<dbl>
D9_profile,D9_profile,control,13.25
D99_profile,D99_profile,control,17.75
D98_profile,D98_profile,control,6.25


In [35]:
PCs_status <- merge(x=id_diagnosis, y=ids_pcs, by.x='ID', by.y='IDs',sort=F)
PCs_status[1:3,]

,ID,Diagnosis,Calprotectin,PC1,PC2,PC3,PC4
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,D9_profile,control,13.25,18.408583,-16.338130,13.1128406,-0.1710436
2,D99_profile,control,17.75,-5.756742,-9.547467,-16.9077999,11.9573567
3,D98_profile,control,6.25,4.608204,16.621946,0.1579095,1.7185819


In [42]:
# export PCs computed using bray distances:
write.table(PCs_status,file="./PCs_status_clr.txt",quote=F,row.names=F,sep="\t")
